In [1]:
#imports
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from imblearn.over_sampling import SMOTE 
import numpy as np

import keras
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.utils import to_categorical

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
#Models: Neural Net, SVM, Random Forest
#train equal amounts of fraud and not fraud data
#neural net: ? number of fully connected layers - softmax activation at the end, relu in the middle, adam optimizer, could use dropout, also use batch norm
#SVM: we can choose kernel/degree (3), slack/seed, penalized SVM 
#random forest: 

In [3]:
data = pd.read_csv('creditcard.csv')

In [4]:
data = data.drop('Time', 1)

In [6]:
data = data.as_matrix()

In [7]:
y = data[ : ,data.shape[1]-1:data.shape[1]]
X = data[:, 0:data.shape[1]-1]

In [8]:
#we dropped the time column 
#maybe time might matter - for mini rapid transactions?

In [9]:
X = preprocessing.scale(X) #perform feature normalization - subtract mean divide by standard deviation

In [10]:
sm = SMOTE(random_state=42)
X, y = sm.fit_sample(X, y)

C:\Users\Murtaza\Anaconda3\envs\mlab\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
#split using sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
#one-hot encode the y vectors:
y_train_cat = to_categorical(y_train, num_classes=2)

In [13]:
split = int(X_train.shape[0]*.8)
x_training = X_train[:split]
x_validation = X_train[split:]
y_training = y_train_cat[:split]
y_validation = y_train_cat[split:]

In [14]:
#Neural Net - Murtaza
#SVM - Naren
#Random Forest - Rohan

In [19]:
model = Sequential()

In [20]:
model.add(Dense(16, input_shape=(29,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.2))

In [21]:
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [22]:
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.5))

In [23]:
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.5))

In [24]:
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.5))

In [25]:
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [26]:
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(.4))

In [27]:
model.add(Dense(2))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                480       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
__________

In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
class_weights= {0:1, 1:100}

In [31]:
model.fit(x_training, y_training, batch_size=64, verbose=2, epochs=13, class_weight=class_weights)

Epoch 1/13
68s - loss: 4.9653 - acc: 0.6784
Epoch 2/13
42s - loss: 1.1787 - acc: 0.7201
Epoch 3/13
36s - loss: 0.9622 - acc: 0.7774
Epoch 4/13
34s - loss: 0.8610 - acc: 0.8109
Epoch 5/13
35s - loss: 0.7793 - acc: 0.8334
Epoch 6/13
36s - loss: 0.7250 - acc: 0.8469
Epoch 7/13
34s - loss: 0.6776 - acc: 0.8615
Epoch 8/13
34s - loss: 0.6597 - acc: 0.8646
Epoch 9/13
34s - loss: 0.6400 - acc: 0.8697
Epoch 10/13
34s - loss: 0.6386 - acc: 0.8720
Epoch 11/13
34s - loss: 0.6350 - acc: 0.8747
Epoch 12/13
34s - loss: 0.5976 - acc: 0.8802
Epoch 13/13
34s - loss: 0.5838 - acc: 0.8841


In [32]:
y_scores = model.predict(x_validation)

In [33]:
def convertProbsToClass(y_scores):
    l = []
    for i in range(y_scores.shape[0]):
        val1 = y_scores[i][0]
        val2 = y_scores[i][1]
        if val1 >= val2:
            l.append(0)
        elif val2 > val1:
            l.append(1)
    return np.array(l)
    
# 

In [34]:
ys = convertProbsToClass(y_scores)
y_actual = convertProbsToClass(y_validation)
print(average_precision_score(y_actual, ys))

0.936187777732
